<a href="https://colab.research.google.com/github/RadwaFathi/ImageProcessing-ComputerVision/blob/main/ImageProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import needed libraries

In [ ]:
#add imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive
import gdown
from moviepy.editor import ImageSequenceClip

  if event.key is 'enter':



# 1. Video Collection

In [ ]:
# Download video from Google Drive
shared_link = "https://drive.google.com/file/d/1FlF50_uqXuBCRx1Mu5FaCvxUwz5EqWEs/view?usp=sharing"
file_id = shared_link.split("/d/")[1].split("/view")[0]
download_url = f"https://drive.google.com/uc?id={file_id}"
video_path = "Computer_Vision_Car_Video.mp4"
gdown.download(download_url, video_path, quiet=False)
print(f"Video downloaded successfully as {video_path}")

Downloading...
From: https://drive.google.com/uc?id=1FlF50_uqXuBCRx1Mu5FaCvxUwz5EqWEs
To: /content/Computer_Vision_Car_Video.mp4
100%|██████████| 3.03M/3.03M [00:00<00:00, 49.6MB/s]

Video downloaded successfully as Computer_Vision_Car_Video.mp4


# 2. Video Preprocessing

In [ ]:
# Create directory for frames
frames_dir = "/content/frames"
os.makedirs(frames_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
 # Extract at least 20 frames per second

frame_interval = int(fps / 20)
frame_count = 0
# Store some selected frames for further processing
selected_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        #convert frames to gray scale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        frame_filename = os.path.join(frames_dir, f"frame_{frame_count}.jpg")

        #save frames to a directory
        cv2.imwrite(frame_filename, gray_frame)
        selected_frames.append(gray_frame)

    frame_count += 1

cap.release()
print(f"Extracted {len(selected_frames)} frames.")

Extracted 120 frames.


# 3. Apply Image Processing Techniques

Histogram Analysis:

In [ ]:
# Store histogram visualization frames
histogram_frames = []
histogram_output_dir = "histogram_analysis"
os.makedirs(histogram_output_dir, exist_ok=True)

for i, gray_frame in enumerate(selected_frames):
    # Create histogram visualization
    plt.figure(figsize=(10, 4))
    plt.hist(gray_frame.ravel(), bins=256, range=[0,256], color='gray', alpha=0.7)
    plt.title(f"Histogram of Frame {i}")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")

    # Save the histogram plot to an image file
    hist_filename = os.path.join(histogram_output_dir, f"hist_{i:04d}.png")
    plt.savefig(hist_filename)
    plt.close()

    # Read back the saved image and convert to OpenCV format
    hist_img = cv2.imread(hist_filename)
    histogram_frames.append(hist_img)

    # Create side-by-side comparison (original frame + histogram)
    frame_colored = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    frame_resized = cv2.resize(frame_colored, (hist_img.shape[1], hist_img.shape[0]))
    comparison = np.hstack((frame_resized, hist_img))

    # Save the comparison image
    comparison_filename = os.path.join(histogram_output_dir, f"compare_{i:04d}.jpg")
    cv2.imwrite(comparison_filename, comparison)

print(f"Histogram analysis applied to {len(selected_frames)} frames.")
print(f"Histogram images saved in {histogram_output_dir} directory")

Histogram analysis applied to 120 frames.
Histogram images saved in histogram_analysis directory


Histogram Equalization:

In [ ]:
# Store equalized frames
equalized_frames = []
comparison_output_dir = "equalized_comparison"
os.makedirs(comparison_output_dir, exist_ok=True)

for i, gray_frame in enumerate(selected_frames):
    equalized = cv2.equalizeHist(gray_frame)
    equalized_frames.append(equalized)

    # Stack original and equalized side by side for comparison
    comparison = np.hstack((gray_frame, equalized))

    # Save the side-by-side image
    comparison_filename = os.path.join(comparison_output_dir, f"compare_{i:04d}.jpg")
    cv2.imwrite(comparison_filename, comparison)

print(f"Histogram equalization applied to {len(equalized_frames)} frames.")
print(f"Comparison images saved in {comparison_output_dir}")


Histogram equalization applied to 120 frames.
Comparison images saved in equalized_comparison


Fourier Transform:

In [ ]:
fourier_output_dir = "fourier_results"
os.makedirs(fourier_output_dir, exist_ok=True)

reconstructed_frames = []

for i, frame in enumerate(selected_frames):
    # 1. Apply 2D Fourier Transform
    f = np.fft.fft2(frame)
    fshift = np.fft.fftshift(f)
    # Add 1 to avoid log(0)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)

    # 2. Apply a Low-Pass Filter (removing high frequencies)
    rows, cols = frame.shape
    crow, ccol = rows // 2 , cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    # Tune this for more/less filtering
    radius = 50
    cv2.circle(mask, (ccol, crow), radius, 1, thickness=-1)

    # 3. Filter and Inverse FT
    fshift_filtered = fshift * mask
    f_ishift = np.fft.ifftshift(fshift_filtered)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back).astype(np.uint8)

    reconstructed_frames.append(img_back)

    # 4. Save visualizations
    # Comparison Original | Spectrum | Reconstructed
    spectrum_normalized = cv2.normalize(magnitude_spectrum, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    stacked = np.hstack((frame, spectrum_normalized, img_back))
    output_path = os.path.join(fourier_output_dir, f"fourier_{i:04d}.jpg")
    cv2.imwrite(output_path, stacked)

print(f"Fourier transform applied to {len(selected_frames)} frames.")
print(f"Results saved in {fourier_output_dir}")


Fourier transform applied to 120 frames.
Results saved in fourier_results


Edge Detection:

In [ ]:
edge_output_dir = "edges_with_original"
edge_output_dir2 = "edges"
os.makedirs(edge_output_dir, exist_ok=True)
os.makedirs(edge_output_dir2, exist_ok=True)

# Define Prewitt kernels
prewitt_kernel_x = np.array([[1, 0, -1],
                             [1, 0, -1],
                             [1, 0, -1]], dtype=np.float32)

prewitt_kernel_y = np.array([[1, 1, 1],
                             [0, 0, 0],
                             [-1, -1, -1]], dtype=np.float32)

for i, frame in enumerate(selected_frames):
    # Apply Canny
    canny_edges = cv2.Canny(frame, 100, 200)

    # Apply Sobel
    sobelx = cv2.Sobel(frame, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(frame, cv2.CV_64F, 0, 1, ksize=3)
    sobel_combined = cv2.magnitude(sobelx, sobely)
    sobel_combined = np.uint8(np.clip(sobel_combined, 0, 255))

    # Apply Prewitt
    prewitt_x = cv2.filter2D(frame, -1, prewitt_kernel_x)
    prewitt_y = cv2.filter2D(frame, -1, prewitt_kernel_y)
    prewitt_combined = cv2.magnitude(prewitt_x.astype(np.float32), prewitt_y.astype(np.float32))
    prewitt_combined = np.uint8(np.clip(prewitt_combined, 0, 255))

    # Comparison Original | Canny | Sobel | Prewitt
    stacked_edges = np.hstack((frame, canny_edges, sobel_combined, prewitt_combined))

    #save edges with the orignial image
    edge_path = os.path.join(edge_output_dir, f"edges_with_original_{i:04d}.jpg")
    cv2.imwrite(edge_path, stacked_edges)

    #save images with edges only
    edge_path2 = os.path.join(edge_output_dir2, f"edges_{i:04d}.jpg")
    cv2.imwrite(edge_path2, canny_edges)


print(f"Applied Canny, Sobel, and Prewitt edge detection on {len(selected_frames)} frames.")
print(f"Stacked comparison images saved in '{edge_output_dir}' folder.")


Applied Canny, Sobel, and Prewitt edge detection on 120 frames.
Stacked comparison images saved in 'edges_with_original' folder.


Kernel Filtering:

In [ ]:
kernel_filter_output_dir = "kernel_filtered"
os.makedirs(kernel_filter_output_dir, exist_ok=True)

for i, frame in enumerate(selected_frames):
    # 1. Apply Gaussian Filter first (noise reduction)
    gaussian_filtered = cv2.GaussianBlur(frame, (5, 5), 0)

    # 2. Apply Bilateral Filter on the Gaussian-filtered result (edge-preserving smoothing)
    combined_filtered = cv2.bilateralFilter(gaussian_filtered, 9, 75, 75)

    # 3. Compare: Original | Gaussian Only | Combined (Gaussian + Bilateral)
    stacked_filtered = np.hstack((frame, gaussian_filtered, combined_filtered))

    # Save the comparison images
    filter_output_path = os.path.join(kernel_filter_output_dir, f"filtered_{i:04d}.jpg")
    cv2.imwrite(filter_output_path, stacked_filtered)

print(f"Combined Gaussian + Bilateral filters applied to {len(selected_frames)} frames.")
print(f"Results saved in '{kernel_filter_output_dir}' folder.")


Combined Gaussian + Bilateral filters applied to 120 frames.
Results saved in 'kernel_filtered' folder.


# **BONUS**

Highlight possible lane markings or object boundaries.

Use contour detection for precise object segmentation.


In [ ]:
# Directory to save output frames with lane markings and object boundaries
output_dir = "lane_and_object_boundaries"
os.makedirs(output_dir, exist_ok=True)

for i, frame in enumerate(selected_frames):
    # Apply Canny Edge Detection (if not done already)
    edges = cv2.Canny(frame, 100, 200)

    # Apply Hough Transform to detect lanes (lines)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=10)
    # Make a copy of the original frame to highlight lanes
    lane_markings = frame.copy()

    # Draw the lines (lane markings) in green
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(lane_markings, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green lines

    # Apply Contour Detection for object boundaries
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
     # Make another copy of the original frame for contours
    contour_output = frame.copy()

    # Draw the contours (object boundaries) in red
    for contour in contours:
      # Filter out small contours
        if cv2.contourArea(contour) > 100:
            cv2.drawContours(contour_output, [contour], -1, (0, 0, 255), 2)  # Red contours

    # Stack the images for visual comparison: Original | Lane Markings | Object Boundaries
    stacked_output = np.hstack((frame, lane_markings, contour_output))

    # Save the output frame
    output_path = os.path.join(output_dir, f"frame_{i:04d}.jpg")
    cv2.imwrite(output_path, stacked_output)

print(f"Lane markings and object boundaries highlighted and saved in '{output_dir}' folder.")


Lane markings and object boundaries highlighted and saved in 'lane_and_object_boundaries' folder.


Incorporate Hough Transform HT for robust lane detection.

In [ ]:
# Robust Lane Detection with Hough Transform
lane_output_dir = "lane_detection_ht"
os.makedirs(lane_output_dir, exist_ok=True)

for i, frame in enumerate(selected_frames):
    # 1. Preprocessing
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # 2. Create ROI mask (focus on lower half for lanes)
    height, width = edges.shape
    mask = np.zeros_like(edges)
    vertices = np.array([[
        (width * 0.1, height),  # Bottom-left
        (width * 0.45, height * 0.6),  # Top-left
        (width * 0.55, height * 0.6),  # Top-right
        (width * 0.9, height)  # Bottom-right
    ]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # 3. Hough Transform for line detection
    lines = cv2.HoughLinesP(masked_edges,
                           rho=1,
                           theta=np.pi/180,
                           threshold=30,
                           minLineLength=40,
                           maxLineGap=25)

    # 4. Process and classify lines
    left_lines = []
    right_lines = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]

            # Calculate line parameters
            if x2 - x1 != 0:  # Avoid division by zero
                slope = (y2 - y1) / (x2 - x1)
                intercept = y1 - slope * x1

                # Classify as left or right lane
                if slope < -0.5:  # Negative slope = left lane
                    left_lines.append((slope, intercept))
                elif slope > 0.5:  # Positive slope = right lane
                    right_lines.append((slope, intercept))

    # 5. Average and extrapolate lanes
    def average_lines(lines, y_min, y_max):
        if not lines:
            return None

        slopes, intercepts = zip(*lines)
        avg_slope = np.mean(slopes)
        avg_intercept = np.mean(intercepts)

        # Calculate x-coordinates
        x1 = int((y_min - avg_intercept) / avg_slope)
        x2 = int((y_max - avg_intercept) / avg_slope)

        return ((x1, y_min, x2, y_max))

    y_min = int(height * 0.6)
    y_max = height

    left_lane = average_lines(left_lines, y_min, y_max)
    right_lane = average_lines(right_lines, y_min, y_max)

    # 6. Visualization
    lane_img = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)

    # Draw ROI area
    cv2.polylines(lane_img, [vertices], True, (0, 255, 255), 1)

    # Draw raw edges
    lane_img[masked_edges > 0] = [0, 0, 255]  # Red edges

    # Draw detected lanes
    if left_lane:
        cv2.line(lane_img, (left_lane[0], left_lane[1]),
                         (left_lane[2], left_lane[3]), (0, 255, 0), 3)
    if right_lane:
        cv2.line(lane_img, (right_lane[0], right_lane[1]),
                         (right_lane[2], right_lane[3]), (0, 255, 0), 3)

    # Draw lane area (if both lanes detected)
    if left_lane and right_lane:
        lane_area = np.array([
            [left_lane[0], left_lane[1]],
            [left_lane[2], left_lane[3]],
            [right_lane[2], right_lane[3]],
            [right_lane[0], right_lane[1]]
        ])
        cv2.fillPoly(lane_img, [lane_area], (0, 100, 0))

    # Save output
    output_path = os.path.join(lane_output_dir, f"lane_{i:04d}.jpg")
    cv2.imwrite(output_path, lane_img)

print(f"Robust lane detection applied to {len(selected_frames)} frames.")
print(f"Results saved in '{lane_output_dir}' folder.")

Robust lane detection applied to 120 frames.
Results saved in 'lane_detection_ht' folder.


# How These Techniques Could Be Used in an Actual Autonomous Vehicle System?

1)Lane Detection (Hough Transform):
The Hough Transform identifies lane markings, helping the vehicle stay centered in its lane and aiding in lane-keeping assistance.

2)Object Boundary Detection (Contours):
Detecting object boundaries (e.g., other vehicles, pedestrians) is crucial for collision avoidance, allowing the vehicle to take appropriate actions like slowing down or steering.

3)Edge Preservation and Noise Reduction:
Techniques like bilateral filtering reduce noise while preserving edges, ensuring clear lane markings and objects in low-contrast conditions (e.g., rain or night).

4)Region of Interest (ROI):
Focusing on key areas (like lane detection) optimizes processing and improves accuracy, especially in the central region where the vehicle is focused.

5)Real-Time Processing:
Efficient edge detection and filtering enable the vehicle to quickly analyze its environment and make real-time decisions for lane-keeping and obstacle avoidance.

# **Making the videos for each technique**

In [ ]:
# Create Videos from Processed Frames
# Define video parameters
video_fps = 20  # Frames per second
video_size = (640, 480)  # Unified size for all videos

# Create videos for each processing step
processing_steps = [
    ("histogram_analysis", "histogram_comparison.mp4"),
    ("equalized_comparison", "histogram_equalization.mp4"),
    ("fourier_results", "fourier_transform.mp4"),
    ("edges", "edge_detection.mp4"),
    ("kernel_filtered", "kernel_filtering.mp4"),
    ("lane_and_object_boundaries", "lane_object_detection.mp4"),
    ("lane_detection_ht", "lane_detection_hough.mp4"),
    ("edges_with_original","edges_with_original.mp4")
    #,
   # ("object_segmentation", "object_segmentation.mp4")
]

for input_dir, output_video in processing_steps:
    image_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir)
                        if f.endswith(('.jpg', '.png'))])

    if not image_files:
        continue

    # Create video writer with unified size
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, video_fps, video_size)

    for image_file in image_files:
        img = cv2.imread(image_file)
        if img is not None:
            img_resized = cv2.resize(img, video_size)
            video_writer.write(img_resized)

    video_writer.release()
    print(f"Created: {output_video}")

# Create combined video with proper sizing
print("\nCreating combined processing video...")

# Define combined video parameters
panel_width = 320  # Each panel width
panel_height = 240  # Each panel height
combined_size = (panel_width * 4, panel_height * 2)  # 4x2 grid (1280x480)

processing_steps = [step[0] for step in processing_steps]
all_images = {step: sorted(os.listdir(step)) for step in processing_steps}
min_frames = min(len(images) for images in all_images.values())

# Create blank image template for missing frames
blank_panel = np.zeros((panel_height, panel_width, 3), dtype=np.uint8)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
combined_writer = cv2.VideoWriter("combined_processing.mp4", fourcc, video_fps, combined_size)

for i in range(min_frames):
    rows = []
    for row in range(2):
        current_row = []
        for col in range(4):
            step_idx = row * 4 + col
            if step_idx >= len(processing_steps):
                # Add blank panel if no more steps
                current_row.append(blank_panel)
                continue

            step = processing_steps[step_idx]
            img_path = os.path.join(step, all_images[step][i])
            img = cv2.imread(img_path)

            if img is not None:
                img_resized = cv2.resize(img, (panel_width, panel_height))
                current_row.append(img_resized)
            else:
                # Add blank panel if image loading fails
                current_row.append(blank_panel)

        # Ensure exactly 4 panels per row
        row_stack = np.hstack(current_row)
        rows.append(row_stack)

    # Combine rows vertically
    final_frame = np.vstack(rows)

    # Verify final dimensions before writing
    if final_frame.shape[1] == combined_size[0] and final_frame.shape[0] == combined_size[1]:
        combined_writer.write(final_frame)
    else:
        print(f"Skipping frame {i} due to incorrect dimensions: {final_frame.shape}")

combined_writer.release()
print("Combined video created successfully!")

Created: histogram_comparison.mp4
Created: histogram_equalization.mp4
Created: fourier_transform.mp4
Created: edge_detection.mp4
Created: kernel_filtering.mp4
Created: lane_object_detection.mp4
Created: lane_detection_hough.mp4
Created: edges_with_original.mp4

Creating combined processing video...
Combined video created successfully!


In [ ]:
def load_processed_frames(directory):
    frames = []
    files = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg','.png'))])
    for f in files:
        img = cv2.imread(os.path.join(directory, f))
        if img is not None:
            frames.append(img)
    return frames

# Load the three techniques
lane_frames = load_processed_frames("lane_detection_ht")
filter_frames = load_processed_frames("kernel_filtered")
edge_frames = load_processed_frames("edges")

# Video parameters
output_fps = 20
min_frames = min(len(lane_frames), len(filter_frames), len(edge_frames))
composite_frames = []

# Standard dimensions for each technique's output
tech_width = 640
tech_height = 240  # Fixed height for each technique section
composite_width = 1280
composite_height = tech_height * 3 + 100  # 3 sections + padding

for i in range(min_frames):
    # Create blank canvas (black background)
    composite = np.zeros((composite_height, composite_width, 3), dtype=np.uint8)

    # 1. Lane Detection (top section)
    lane_img = lane_frames[i]
    lane_resized = cv2.resize(lane_img, (tech_width, tech_height))
    composite[20:20+tech_height, (composite_width-tech_width)//2:(composite_width-tech_width)//2+tech_width] = lane_resized

    # 2. Kernel Filtering (middle section)
    filter_full = filter_frames[i]
    # Extract just the filtered portion (rightmost panel)
    filter_img = filter_full[:, filter_full.shape[1]//3*2:]
    filter_resized = cv2.resize(filter_img, (tech_width, tech_height))
    y_offset = 20 + tech_height + 20
    composite[y_offset:y_offset+tech_height, (composite_width-tech_width)//2:(composite_width-tech_width)//2+tech_width] = filter_resized

    # 3. Edge Detection (bottom section)
    edge_full = edge_frames[i]
    # Extract just the Canny edges portion (second panel)
    edge_img = edge_full[:, edge_full.shape[1]//4:edge_full.shape[1]//4*2]
    edge_resized = cv2.resize(edge_img, (tech_width, tech_height))
    y_offset = y_offset + tech_height + 20
    composite[y_offset:y_offset+tech_height, (composite_width-tech_width)//2:(composite_width-tech_width)//2+tech_width] = edge_resized

    # Add labels
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(composite, "Lane Detection", (50, 15), font, 0.7, (255,255,255), 2)
    cv2.putText(composite, "Kernel Filtering", (50, 20 + tech_height + 15), font, 0.7, (255,255,255), 2)
    cv2.putText(composite, "Edge Detection (Canny)", (50, 20 + tech_height*2 + 35), font, 0.7, (255,255,255), 2)

    composite_frames.append(cv2.cvtColor(composite, cv2.COLOR_BGR2RGB))

# Create final video
final_clip = ImageSequenceClip(composite_frames, fps=output_fps)
final_clip.write_videofile(
    "combined_techniques.mp4",
    codec="libx264",
    fps=output_fps,
    preset="medium",
    ffmpeg_params=["-pix_fmt", "yuv420p"],
    threads=4,
    audio=False
)

print("Final video created successfully as 'combined_techniques.mp4'")

Moviepy - Building video combined_techniques.mp4.
Moviepy - Writing video combined_techniques.mp4



Moviepy - Done !
Moviepy - video ready combined_techniques.mp4
Final video created successfully as 'combined_techniques.mp4'
